## Custom Tutorial 3: Using your model

In this tutorial, we demonstrate how to fit your custom model to data. The process is nearly identical to that of the default sub-module, so we will principally focus on what's different between the default and custom versions here. Refer to Default Tutorial 3 for use details.

The specific model we will use for this example is the urgency gating model with time depenedent drift discussed in PyBEAM's publication. In this model, the drift rate flips from positive, to negative, then back to positive. The model files for this example are on PyBEAM's github under the folder ugm_drift_flipping.

As before, import PyBEAM's custom sub-module.


In [ ]:
# import PyBEAM's custom module
import pybeam.custom as pbc

# also import pyplot to modify figure axes
import matplotlib.pyplot as plt


Next, we specify the directory where the model is located.


In [ ]:
# the directory containing your model goes here
model_dir = ''

# windows computers
# model_dir = r''


As we did in Custom Tutorial 2, we run a quick check using functions_test to make sure our functions are doing what we expect them to.


In [ ]:
# dictionary containing model parameters
phi = {'phi[0]' : 0.25, # t_nd: non-decision time
       'phi[1]' : 0.5,  # w: relative start point
       'phi[2]' : 1.0,  # mu: drift rate
       'phi[3]' : 3.0,  # l: leakage rate
       'phi[4]' : 2.0,  # k: urgency ratio
       'phi[5]' : 1.0,  # sigma: model scale
       'phi[6]' : 1.0,  # a: threshold location
       'phi[7]' : 0.33, # t0: time for first drift rate flip
       'phi[8]' : 0.66} # t1: time for second drift rate flip
    
pbc.functions_test(model_dir = model_dir, # string containing directory name where your model files are
                         phi = phi,       # dictionary of model parameters
                           x = 0.0,       # accumulator state
                           t = 1.0)       # time


We next simulate two data sets, corresponding to two model conditions. Specifically, we model a speed/accuracy trade-off by simulated data sets with differing threhsold locations. The first has a = 1.0, while the second has a = 1.5


In [ ]:
phi0 = {'phi[0]' : 0.25, # t_nd: non-decision time
        'phi[1]' : 0.5,  # w: relative start point
        'phi[2]' : 1.0,  # mu: drift rate
        'phi[3]' : 3.0,  # l: leakage rate
        'phi[4]' : 2.0,  # k: urgency ratio
        'phi[5]' : 1.0,  # sigma: model scale
        'phi[6]' : 1.0,  # a: threshold location
        'phi[7]' : 0.33, # t0: time for first drift rate flip
        'phi[8]' : 0.66} # t1: time for second drift rate flip

rt0 = pbc.simulate_model(model_dir = model_dir,
                            N_sims = 500,
                               phi = phi0)

pbc.plot_rt(model_dir = model_dir,
                  phi = phi0,
                   rt = rt0,
                 bins = 50);

plt.xlim(-2.5, 2.5)


In [ ]:
phi1 = {'phi[0]' : 0.25, # t_nd: non-decision time
        'phi[1]' : 0.5,  # w: relative start point
        'phi[2]' : 1.0,  # mu: drift rate
        'phi[3]' : 3.0,  # l: leakage rate
        'phi[4]' : 2.0,  # k: urgency ratio
        'phi[5]' : 1.0,  # sigma: model scale
        'phi[6]' : 1.5,  # a: threshold location
        'phi[7]' : 0.33, # t0: time for first drift rate flip
        'phi[8]' : 0.66} # t1: time for second drift rate flip

rt1 = pbc.simulate_model(model_dir = model_dir,
                            N_sims = 500,
                               phi = phi1)

pbc.plot_rt(model_dir = model_dir,
                  phi = phi1,
                   rt = rt1,
                 bins = 50);

plt.xlim(-2.5, 2.5)


Now that we have data, we can infer what parameters best describe the data set. This follows the same process as the default sub-module, discussed in Default Tutorial 3. We first define a bank of priors in dictionary p. Then, we define our condition dictionaries. Note that, instead of parameter names for keys, we write the array location 'phi[0]', 'phi[1]', ... , 'phi[N_phi-1]'. This is as we did for the phi dictionary inputs to the simulate and plot model functions. These sub-dictionaries are placed into another condition dictionary in the same way as for the default model.

Now that we have done this, we can call the inference function. This is identical the callout used in the default tutorial other than the model input. As with the other custom functions, we define the model directory instead of a model dictionary.

The analysis tools plot_trace and summary are the same as for the default module.


In [ ]:
p = {'pphi[0]' : 'Uniform("t_nd", lower = 0.0, upper = 0.75)', # non-decision time prior
     'pphi[1]' : 'Uniform("w", lower = 0.3, upper = 0.7)',     # relative start point prior
     'pphi[2]' : 'Uniform("mu", lower = -5.0, upper = 5.0)',   # drift rate prior
     'pphi[3]' : 'Uniform("l", lower = 0.0, upper = 10.0)',    # leakage rate prior
     'pphi[4]' : 'Uniform("k", lower = 0.0, upper = 10.0)',    # urgency rate prior
     'pphi[5]' : 1.0,                                          # scaling parameter
     'pphi[6]0' : 'Uniform("a0", lower = 0.25, upper = 3.0)',  # decision threshold prior 0
     'pphi[6]1' : 'Uniform("a1", lower = 0.25, upper = 3.0)',  # decision threshold prior 1
     'pphi[7]' : 0.33,                                         # first drift flip time
     'pphi[8]' : 0.66}                                         # second drift flip time

c0 = {'rt' : rt0,
  'phi[0]' : 'pphi[0]', 
  'phi[1]' : 'pphi[1]',  
  'phi[2]' : 'pphi[2]', 
  'phi[3]' : 'pphi[3]',  
  'phi[4]' : 'pphi[4]',  
  'phi[5]' : 'pphi[5]',
  'phi[6]' : 'pphi[6]0', 
  'phi[7]' : 'pphi[7]',  
  'phi[8]' : 'pphi[8]'}  

c1 = {'rt' : rt1,
  'phi[0]' : 'pphi[0]', 
  'phi[1]' : 'pphi[1]',  
  'phi[2]' : 'pphi[2]', 
  'phi[3]' : 'pphi[3]',  
  'phi[4]' : 'pphi[4]',  
  'phi[5]' : 'pphi[5]',
  'phi[6]' : 'pphi[6]1', 
  'phi[7]' : 'pphi[7]',  
  'phi[8]' : 'pphi[8]'}  

cond = {0 : c0, 1 : c1}

trace = pbc.inference(model_dir = model_dir, # specify model directory
                         priors = p,         # dictionary of priors
                     conditions = cond,      # conditions dictionary
                        samples = 50000,     # MCMC samples
                         chains = 3,         # MCMC chains 
                          cores = 3,         # CPU cores to run MCMC chains on
                      file_name = 'custom')  # output file name


In [ ]:
# summary of posteriors
pbc.plot_trace(file_name = 'custom', burnin = 25000);


In [ ]:
# summary of posteriors
pbc.summary(file_name = 'custom', burnin = 25000)
